# Data exploration
This notebooks provides tools to explore the datasets at hand. It can be useful to detect potential bias in data.

In [3]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/P2
!ls

/content/drive/.shortcut-targets-by-id/19-yFNaBIFlRy843OcbnYrC9ElMCzmXbA/P2
data  Data_exploration.ipynb  model_pneumonia.ipynb  pneumonia_bacterial.ipynb


In [6]:
username = 'ColinPelletier'
repository = 'CS-433/ml-project-2-la_team'
git_token = 'TODO_SET_ME'

In [7]:
!git clone https://{git_token}@github.com/{repository}

Cloning into 'ml-project-2-la_team'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 117 (delta 0), reused 12 (delta 0), pack-reused 104
Receiving objects: 100% (117/117), 152.74 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Checking out files: 100% (35/35), done.


In [14]:
%cd ml-project-2-la_team/
!ls

/content/drive/.shortcut-targets-by-id/19-yFNaBIFlRy843OcbnYrC9ElMCzmXbA/P2/ml-project-2-la_team
data
generated
metrics.py
notebooks
pneumonia-detection-using-cnn-92-6-accuracy.ipynb
README.md
requirements.txt
res
results
script_change_images.py
src
temp
tensorflow-pneumonia-classification-on-x-rays.ipynb


In [18]:
!git checkout -b adapt_3_kind_pneumonia

fatal: A branch named 'adapt_3_kind_pneumonia' already exists.


In [19]:
!git pull origin adapt_3_kind_pneumonia

From https://github.com/CS-433/ml-project-2-la_team
 * branch            adapt_3_kind_pneumonia -> FETCH_HEAD
Updating a02dd14..d9a52ab
Checking out files: 100% (22/22), done.
Fast-forward
 generated/3_kind_pneumonia/run_01/history.txt      |    1 +
 .../3_kind_pneumonia/run_01/history_finetune.txt   |    1 +
 .../run_01/model-20221130T223902Z-001.zip          |  Bin 0 -> 39498805 bytes
 generated/3_kind_pneumonia/run_01/xray_model.h5    |  Bin 0 -> 42146120 bytes
 generated/{ => x_rays_dataset}/run_01/history.txt  |    0
 .../run_01/history_finetune.txt                    |    0
 .../{ => x_rays_dataset}/run_01/keras_model_1.zip  |  Bin
 generated/{ => x_rays_dataset}/run_01/place_holder |    0
 .../{ => x_rays_dataset}/run_01/xray_model.h5      |  Bin
 metrics.py                                         |   76 -
 notebooks/model.ipynb                              | 1734 ++++++++++++++++++++
 ...eumonia-detection-using-cnn-92-6-accuracy.ipynb |    0
 src/metrics.py                     

In [25]:
!git config --global user.email "colinpelletier@outlook.com"
!git config --global user.name "ColinPelletier"

In [30]:
!git add --all
!git status

On branch adapt_3_kind_pneumonia
Your branch is up to date with 'origin/adapt_3_kind_pneumonia'.

nothing to commit, working tree clean


In [32]:
!ls notebooks

main.ipynb  model.ipynb  pneumonia-detection-using-cnn-92-6-accuracy.ipynb


In [33]:
# clear cash and update file changes
!google-drive-ocamlfuse -cc

/bin/bash: google-drive-ocamlfuse: command not found


In [27]:
# commit
!git add .
!git commit -m "first push from Google Colab"
!git push --set-upstream origin adapt_3_kind_pneumonia

On branch adapt_3_kind_pneumonia
nothing to commit, working tree clean
Branch 'adapt_3_kind_pneumonia' set up to track remote branch 'adapt_3_kind_pneumonia' from 'origin'.
Everything up-to-date
